In [1]:
from cnn_method import *
from cnn_preproc_function import *
from cnn_inference_method import *
from inference_function import *

import optparse
import numpy as np
import pandas as pd

import sklearn
from sklearn import preprocessing

import keras
import tensorflow as tf

from keras_model_configuration import *
from keras_metric import *

import datetime
import time
import os
import shutil
from sklearn.externals import joblib

assetindex = int(0)
gpudevice = "0"

#GPU CONFIG
os.environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"] = "4"
os.environ["CUDA_VISIBLE_DEVICES"]=gpudevice
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

Using TensorFlow backend.


In [5]:
#SETUP INIT
main_folder = "/home/workstation/Desktop/CNN/output/no_scale_models"
# main_folder = "afp://cxtanalytics@Alpha.local/CXT%20Analytics/ML%20Training%20Data/_CNN_Output"
loss_criteria = "precision"
loss_mode = "max"
batch_size = 64
daily = False

try:
    setups_folder = [j for j in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder,j))]
    print(setups_folder)
    setup_folder_name = setups_folder[assetindex]
    print("Running:", setup_folder_name)

except Exception as err:
    print("Index:", assetindex, "--SETUP ERROR!")
    raise Exception(err)


['XLV_1540132613', 'VWO_1540124794', 'EURJPY_1540090805', 'XOP_1540119549', 'EURGBP_1540081332', 'TLT_1540096938', 'USDCAD_1540071829', 'EEM_1540128128', 'IYR_1540143733', 'USO_1540109072', 'GLD_1540146336', 'EWT_1540148621', 'EURUSD_1540071836', 'EWH_1540100271', 'XLK_1540114315', 'EWZ_1540102904', 'XLY_1540137805', 'SLV_1540081303', 'EWW_1540143008', 'USDJPY_1540154373', 'XRT_1540135209', 'EFA_1540130727', 'XLE_1540127415', 'XLU_1540141127', 'XBI_1540145600', 'SMH_1540091698', 'SPY_1540138520', 'USDCHF_1540099565', 'OIH_1540089114', 'IEF_1540150057', 'LQD_1540116041', 'XLI_1540130012', 'RSX_1540083885', 'FXI_1540105534', 'GBPUSD_1540062289', 'EWJ_1540086498', 'AGG_1540151486', 'XLF_1540111692', 'IAU_1540108165', 'NZDUSD_1540118665', 'VNQ_1540140403', 'HYG_1540135925', 'GDX_1540133327', 'EZU_1540110791', 'VEA_1540122171', 'UNG_1540152912', 'XLP_1540116940', 'XLB_1540094315', 'KRE_1540113417', 'AUDUSD_1540062289']
Running: XLV_1540132613


In [6]:
#Remove existing signal
# import glob
# for setup_folder_name in setups_folder:
#     try:
#         os.remove(glob.glob(os.path.join(main_folder, setup_folder_name, "*.csv"))[0])
#     except:
#         print("nope", setup_folder_name)

In [7]:
setup_path = os.path.join(main_folder, setup_folder_name)
DATA_PARAMS = joblib.load(os.path.join(setup_path, "DATA_PARAMS.pkl"))
MODEL_PARAMS = joblib.load(os.path.join(setup_path, "MODEL_PARAMS.pkl"))
scaler = joblib.load(os.path.join(setup_path, "scaler.pkl"))
locals().update(DATA_PARAMS)
locals().update(MODEL_PARAMS)

if daily:
    df = load_data_daily_close_missing(raw_data_file)
    df = clean_and_create_target(df, TARGET_TO_PREDICT, FUTURE_PERIOD_PREDICT, TARGET_FUNCTION, TARGET_THRESHOLD, FLIP, False)
else:
    df = load_data(raw_data_file)
    df = clean_and_create_target(df, TARGET_TO_PREDICT, FUTURE_PERIOD_PREDICT, TARGET_FUNCTION, TARGET_THRESHOLD, FLIP, True)

df, timestamp, all_data_gen = FullTSGenerator(df, scaler, batch_size, SEQ_LEN, old = False)


Load Data: Done!


/home/workstation/Desktop/CNN/cnn_preproc_function.py:27: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  df.loc[:,'target'] = df[target_col].rolling(window = FUTURE_PERIOD_PREDICT).apply(lambda x: FUNC(x))
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

Clean Data: Done!


In [8]:
#pick best model
models_folder = os.path.join(setup_path, "models")
mdf = parse_model_folder(models_folder)
fmdf, best_model_file = best_model_score(mdf)
best_model_path = os.path.join(models_folder,best_model_file)
model = keras.models.load_model(best_model_path, custom_objects=None, compile=False)
print("Best Model: Done!")


Best Model: Done!


In [ ]:
t0 = time.time()
y = model.predict_generator(all_data_gen)
t1 = time.time()
print("Prediction: Done!", t1-t0, "seconds!")

signal_df = pd.DataFrame(dict(Date = timestamp, signal_raw = y.flatten())).set_index("Date")
signal_file = "signal_" + TARGET_TO_PREDICT + "_" + str(FLIP) + ".csv"
signal_df.to_csv(os.path.join(setup_path, signal_file))

print("Signal Output: Done!")

In [10]:
shutil.copy2(best_model_path, setup_path)
print("Copy best model: Done!")

Copy best model: Done!
